In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from scipy.stats import pearsonr, spearmanr, pointbiserialr
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from SentenceComparison.SentenceComparison import SentenceComparison
from ModelHelpers.fill_mask_helpers import get_probability_from_pipeline_for_token
from relation_templates.templates import relations, nominalized_relations, get_length_for_relation, get_templates, get_relation_meta, get_relation_name, get_order_for_relation, get_distance_for_relation, starts_with_mask, starts_with_sub

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
#relations = ['P106', 'P108', 'P1303', 'P19', 'P39', 'P413', 'P937'] + ['P103', 'P159', 'P138', 'P136']

In [5]:
logging.set_verbosity_error()

In [6]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(model(sentence), token)
    return prob

In [7]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]
TOP_K = 10

In [8]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations

In [9]:
good_relations = ['P37', 'P1412', 'P178', 'P103', 'P36', 'P407', 'P1376', 'P1001', 'P140', 'P463', 'P176']
middle_relations = ['P159', 'P17', 'P20', 'P276', 'P31', 'P364', 'P30', 'P937', 'P127', 'P279']

In [48]:
considered_relations = good_relations
#considered_relations = middle_relations 

In [11]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
my_tokenizer = lambda sentence: tokenizer(sentence)['input_ids']

In [13]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [14]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 25460.25it/s]


In [15]:
unmasker.save_to_cache()

In [16]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=500)

In [17]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(unmasker(sentence), token)
    return prob

In [18]:
Comparer = SentenceComparison(relations, get_templates, metric, MASK, get_relation_meta)
Comparer.compare(TREx.data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 5708.28it/s]


In [19]:
unmasker.save_to_cache()

In [49]:
accuracies = querier.results_for_calculation()
np_accuracies = np.array(list(filter(lambda row: row[0] in considered_relations, accuracies)))[:,1:].flatten().astype(float)

In [50]:
probabilities = Comparer.make_global_comparison()
np_probabilities = np.array(list(filter(lambda row: row[0] in considered_relations, probabilities)))[:,1:].flatten().astype(float)

In [56]:
lengths = get_length_for_relation(my_tokenizer)
#np_lengths = np.array(list(filter(lambda row: row[0] in considered_relations, starts_with_sub())))[:,1:].flatten().astype(float) 
#np_lengths = np.array(list(filter(lambda row: row[0] in considered_relations, starts_with_mask())))[:,1:].flatten().astype(float)
np_lengths = np.array(list(filter(lambda row: row[0] in considered_relations, lengths)))[:,1:].flatten().astype(float)

# sub, obj -> 1
# obj, sub -> 0

Accuracy vs Length

In [57]:
r_corr , _ = pearsonr(np_accuracies, np_lengths)
r_corr

-0.009251632603120744

In [58]:
r_corr , _ = spearmanr(np_accuracies, np_lengths)
r_corr

-0.02690773757867983

Probability vs Length

In [59]:
r_corr , _ = pearsonr(np_probabilities, np_lengths)
r_corr, r_corr**2*100

(-0.2068695368651644, 4.279500528280762)

In [60]:
r_corr , _ = spearmanr(np_probabilities, np_lengths)
r_corr, r_corr**2*100

(-0.23828862316978058, 5.678146793214969)

### simple vs ...

In [61]:
keys = ["Compound", "Complex", "Compound-Complex"]
key_accuracies = {key:np.array(accuracies)[1:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}
key_probabilities = {key:np.array(probabilities)[:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}
key_lengths = {key:np.array(lengths)[1:,1:].astype(float)[:,[0,index+1]].flatten() for index, key in enumerate(keys)}

In [62]:
for c_key in keys:
    r_ac, _ = pearsonr(key_accuracies[c_key], key_lengths[c_key])
    r_prob, _ = pearsonr(key_probabilities[c_key], key_lengths[c_key])
    print(f"Simple vs {c_key} - accuracy: {r_ac} - probability: {r_prob}")

Simple vs Compound - accuracy: -0.2107830307565549 - probability: -0.21252658243728753
Simple vs Complex - accuracy: -0.1685385159515642 - probability: -0.23071678987748412
Simple vs Compound-Complex - accuracy: -0.26893405224041095 - probability: -0.2127714694283168


In [63]:
for c_key in keys:
    r_ac, _ = spearmanr(key_accuracies[c_key], key_lengths[c_key])
    r_prob, _ = spearmanr(key_probabilities[c_key], key_lengths[c_key])
    print(f"Simple vs {c_key} - accuracy: {r_ac} - probability: {r_prob}")

Simple vs Compound - accuracy: -0.15861853960648492 - probability: -0.31931440635877195
Simple vs Complex - accuracy: -0.13062334191445166 - probability: -0.3532068012282151
Simple vs Compound-Complex - accuracy: -0.18098828543145862 - probability: -0.2719143249112648


In [64]:
for c_key in keys:
    r_ac, _ = pearsonr(key_accuracies[c_key], key_lengths[c_key])
    s_ac, _ = spearmanr(key_accuracies[c_key], key_lengths[c_key])
    print(f"10 & Simple and {c_key} & {r_ac:.4f} & {s_ac:.4f} \\\\")

10 & Simple and Compound & -0.2108 & -0.1586 \\
10 & Simple and Complex & -0.1685 & -0.1306 \\
10 & Simple and Compound-Complex & -0.2689 & -0.1810 \\
